In [1]:
import numpy as np
import pandas as pd

In [2]:
file = pd.read_csv("features_class_csv/kyoto_class_feat_copy.csv").set_index("Date")
del file["Unnamed: 0"]
file.head()


,Description,Avg_temp_1,Avg_temp_2,Avg_temp_3,Avg_temp_4,Avg_temp_5,Avg_temp_6,Avg_temp_7,Avg_temp_8,Avg_temp_9,...,Clouds_63,Clouds_64,Clouds_65,Clouds_66,Clouds_67,Clouds_68,Clouds_69,Clouds_70,Clouds_71,Clouds_72
Date,,,,,,,,,,,,,,,,,,,,,
1/4/17,Clear,3,2,2,-1,-2,-2,-2,-1,-1,...,40,20,20,20,20,20,20,40,75,75
1/4/17,Clouds,4,3,2,2,-1,-2,-2,-2,-1,...,20,40,20,20,20,20,20,20,40,75
1/4/17,Clouds,7,4,3,2,2,-1,-2,-2,-2,...,20,20,40,20,20,20,20,20,20,40
1/4/17,Clouds,9,7,4,3,2,2,-1,-2,-2,...,20,20,20,40,20,20,20,20,20,20
1/4/17,Clouds,11,9,7,4,3,2,2,-1,-2,...,0,20,20,20,40,20,20,20,20,20


In [3]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

/anaconda3/envs/PythonData/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
X = file.loc[:, file.columns != 'Description']
y = file['Description']

In [5]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)


In [6]:
#for label, original_class in zip(encoded_y, y):
    #print('Original Class: ' + str(original_class))
    #print('Encoded Label: ' + str(label))
    #print('-' * 12)

In [7]:
# Split the data into training and testing
from sklearn.model_selection import train_test_split

X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X,encoded_y, random_state=42)


In [8]:
# Convert the dataframe to a numpy array for Keras
X_train = X_train_df.values
X_train.shape

(13028, 360)

In [9]:
# Read in the training labels as a dataframe

# One-hot encode the integer labels
# 0. "Clear" 
# 1. "Clouds"
# 2. "Fog"
# 3. "Haze"
# 4. "Mist"
# 5. "Rain"
# 6. "Snow"
# 7. "Thunderstorm"
#y_train = to_categorical(y_train_df)
y_train = to_categorical(y_train_df)
y_train.shape

(13028, 8)

In [10]:
# Read the testing data
X_test = X_test_df.values
X_test.shape

(4343, 360)

In [11]:
# One-hot encode the integer labels
y_test = to_categorical(y_test_df)
y_test.shape

(4343, 8)

In [12]:
# Create an empty sequential model
model = Sequential()

In [13]:
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(units=1, activation='relu', input_dim=360))
# Add a second hidden layer
model.add(Dense(units=1, activation='relu'))
# The output layer has 8 columns that are one-hot encoded
y_train.shape

(13028, 8)

In [14]:
# Add output layer
model.add(Dense(units=8, activation='softmax'))

In [15]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [16]:
# Use the training data to fit (train) the model
model.fit(
    X_train,
    y_train,
    epochs=1,
    shuffle=True,
    verbose=0
)

ValueError: Error when checking input: expected dense_1_input to have shape (1,) but got array with shape (360,)

In [ ]:
y_pred = model.predict(X_test)
y_pred[0]

In [ ]:
# Save the model
model.save("keras_class_model.h5")

In [ ]:
# Load the model
from keras.models import load_model

voice_model = load_model("keras_class_model.h5")

In [ ]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Grab just one data point to test with
test = np.expand_dims(X_test[0], axis=0)
test.shape

In [ ]:
# Make a prediction. The result should be 3 - Haze
print(f"Predicted class: {model.predict_classes(test)}")
print(f"y_test[0]")